<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/ml_classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from IPython.display import SVG, display, HTML

In [3]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Mounted at ./content


In [195]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [5]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  sample_data
.shortcut-targets-by-id
MyDrive
.file-revisions-by-id
.Trash


In [6]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [7]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv

--2021-05-22 16:51:04--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1162863 (1.1M) [text/plain]
Saving to: ‘coref_df_anns60k.csv’

coref_df_anns60k.cs 100%[===================>]   1.11M  --.-KB/s    in 0.07s   

2021-05-22 16:51:05 (16.5 MB/s) - ‘coref_df_anns60k.csv’ saved [1162863/1162863]

--2021-05-22 16:51:05--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/new_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request s

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [8]:
# with open('coref_df_anns60k.csv') as csvfile:
with open('new_anns60k.csv') as csvfile:
  reader = csv.reader(csvfile, delimiter='\t')
  with open('new_df_anns60k.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    c = 0
    for row in reader:
      new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
      if len(new_row) > 9:
        new_row = new_row[:9]
      writer.writerow(new_row)


In [9]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('new_df_anns60k.csv', delimiter='\t')

Проверим данные

In [10]:
anns_df

,TOKEN,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,IS_ANSWER,HEAD,NER,COREFERENCE_CLUSTER
0,обсуждаем,1.0,0.0,0.0,0.0,0.0,обсуждаем,0.0,0.0
1,препарат,1.0,0.0,0.0,1.0,0.0,обсуждаем,0.0,0.0
2,рисперидон,1.0,0.0,0.0,2.0,0.0,препарат,1.0,2.0
3,рисполепт,1.0,0.0,0.0,3.0,0.0,рисперидон,1.0,3.0
4,сперидан,1.0,0.0,0.0,4.0,0.0,рисполепт,1.0,4.0
...,...,...,...,...,...,...,...,...,...
15405,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15406,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15407,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
15408,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [11]:
import numpy as np

In [12]:
cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k.csv', delimiter=' ')
cor_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
1,препарат,[-0.17725945 0.94834274 0.90867615 0.850609...,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,[-2.08233938e-01 9.96180058e-01 -2.79894114e-...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[-0.17725945 0.94834274 0.90867615 0.850609...,0,0
2,рисперидон,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,[-0.17725945 0.94834274 0.90867615 0.850609...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.15259643 1.0673474 0.84192747 -0.077515...,0,0
3,рисполепт,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,[ 0.15259643 1.0673474 0.84192747 -0.077515...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,0,0
4,сперидан,[ 0.08634768 0.893356 0.55127096 -0.065075...,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,[ 1.2426247e-02 1.0408522e+00 6.9133490e-01 ...,[ 0.01863711 0.2306802 0.14600733 -0.019119...,[ 0.02878256 0.29778534 0.18375699 -0.021691...,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26492,и,[ 0.5048941 0.81449467 0.29763442 0.603937...,0,22,68,0,7,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26493,таможенные,[ 0.73242044 -0.24206167 -0.6556247 -0.476650...,0,22,68,0,8,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 0.41243815 0.190811 -0.11933009 0.042429...,0,0
26494,пошлины,[ 0.7365944 -0.48505062 -0.00257558 0.168963...,0,22,68,0,9,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0
26495,как,[ 1.0349948e+00 7.3793733e-01 -3.8374668e-01 ...,0,22,68,0,10,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,[-0.0050342 0.14796825 0.08416066 -0.776544...,[ 1.00772209e-01 6.29534647e-02 -3.53814550e-...,[ 5.7842094e-01 9.1962010e-02 -1.9555457e-01 ...,0,0


In [13]:
cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

0.8436395154681279

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [14]:
cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]

In [15]:
cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]

In [16]:
cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]

In [17]:
cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]

Добавим размеченные данные, и посмотрим, что получилось.

In [18]:
cor_df['NER'] = anns_df['NER']
cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [227]:
df = cor_df.iloc[:9500,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [228]:
len(df.loc[df['NER'] == 1])

369

Из них местоимений (прономинальные анафоры).

In [229]:
len(df.loc[df['NER'] == 3])

74

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [230]:
len(df.loc[df['NER'] == 2])

126

In [231]:
len(df.loc[df['NER'].isin([1,2,3])])

569

Посмотрим на данные.

In [232]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
3,рисполепт,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...",0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.012426247, 1.0408522, 0.6913349, -0.1638528...",1.0,3.0
4,сперидан,"[0.08634768, 0.893356, 0.55127096, -0.06507572...",0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,"[0.012426247, 1.0408522, 0.6913349, -0.1638528...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.02878256, 0.29778534, 0.18375699, -0.021691...",1.0,4.0
21,лекарством,"[0.598588, -0.07832722, 0.23088205, -0.3704158...",0,1,1,1,10,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,"[1.3479745, 0.34477097, 0.20343542, 0.10220611...","[0.05498165, -0.00549405, 0.01865345, -0.03142...","[0.42120528, 0.21546456, 0.00441866, -0.240917...",2.0,0.0
142,рисполепта,"[0.20826544, 0.5118653, 0.6703367, -0.22502856...",0,1,1,13,7,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,"[0.5131939, 0.9708054, 0.53676724, 0.09788364,...","[0.11585727, 0.1283627, 0.02299715, -0.0095005...","[0.28126505, 0.3790152, 0.5397224, -0.09368, -...",1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8184,он,"[0.7369664, 0.98495233, 0.5467577, -0.1162631,...",1,5,49,2,1982,NaN,nomn,masc,3per,NPRO,nsubj,вообше,inan,loct,masc,NOUN,"[0.8197453, 0.55724543, 0.25136977, 0.22293802...","[0.04829145, -0.00084831, -0.02359735, 0.04864...","[0.456814289, 0.537917078, 0.258037448, -0.032...",3.0,1984.0
8191,ибупрофен,"[0.56447846, 0.32898948, 0.42630774, 0.7194719...",1,5,49,2,1989,NaN,NaN,masc,NaN,ADJS,obl,помогает,NaN,NaN,NaN,VERB,"[0.11804099, -0.17524207, 0.6587767, 0.2620321...","[0.04829145, -0.00084831, -0.02359735, 0.04864...","[0.5514261, 0.3847794, 0.25777692, 0.3411266, ...",1.0,1993.0
8195,них,"[0.698467195, -0.0015053238, -0.428473055, 0.6...",1,5,49,2,1993,NaN,gent,NaN,3per,NPRO,obl,хуже,NaN,NaN,NaN,COMP,"[0.392052829, 0.505600393, 1.10738695, 0.04143...","[0.04829145, -0.00084831, -0.02359735, 0.04864...","[0.52525467, 0.14333361, 0.14554344, 0.3120154...",3.0,0.0
8224,ибупрофен,"[0.56447846, 0.32898948, 0.42630774, 0.7194719...",1,5,50,0,2022,NaN,NaN,masc,NaN,ADJS,nsubj,помогает,NaN,NaN,NaN,VERB,"[0.11804099, -0.17524207, 0.6587767, 0.2620321...","[0.0126089, -0.00790749, 0.04128889, 0.0185373...","[0.627043009, 0.370084018, 0.153539896, 0.4228...",1.0,2026.0


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [233]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,"[0.15259643, 1.0673474, 0.84192747, -0.0775155...",0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01863711, 0.2306802, 0.14600733, -0.0191193...","[0.15259643, 1.0673474, 0.84192747, -0.0775155...",1.0,2.0
576,зелдокс,"[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",0,2,0,0,2,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.03948287, 0.24082112, 0.22588146, -0.017884...","[0.2465947, 1.0097265, 0.31869438, 0.07363212,...",1.0,2.0
1870,оланзапин,"[-0.082683861, 1.2017089, 0.67815673, -0.22599...",0,3,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.01915067, 0.0875298, 0.09080387, 0.00378973...","[-0.082683861, 1.2017089, 0.67815673, -0.22599...",1.0,2.0
5555,галоперидол,"[-0.03019532, 1.2394575, 0.18222564, -0.215572...",0,4,0,0,2,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,"[-0.208233938, 0.996180058, -0.279894114, -0.9...","[-0.03747297, 0.5554195, 0.15133946, -0.011017...","[-0.10372739, 1.0939001, 0.5454509, 0.3175185,...",1.0,2.0
6200,тиаприд,"[0.27581334, 0.80696917, 1.0271364, -0.1037090...",0,5,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[0.10855111, 0.28903672, 0.2550475, -0.0276597...","[0.27581334, 0.80696917, 1.0271364, -0.1037090...",1.0,3.0
8450,сероквель,"[0.087633356, 0.93445277, 0.69432694, 0.298702...",0,6,0,0,2,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,"[-0.17725945, 0.94834274, 0.90867615, 0.850609...","[-0.04207938, 0.36281538, 0.25962716, 0.010672...","[0.087633356, 0.93445277, 0.69432694, 0.298702...",0.0,0.0
8541,день,"[0.11187396, 1.2210127, -0.88301444, -0.466371...",0,7,0,0,2,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,"[0.11187396, 1.2210127, -0.88301444, -0.466371...","[0.10426845, 0.58591, -0.40001512, -0.31153095...","[0.3036255, 0.7870125, -0.33815098, -0.5249253...",0.0,0.0


In [234]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [235]:
token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [236]:
df.TOKEN_VECT[710]

array([nan])

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [237]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [238]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

Посмотрим на данные.

In [239]:
df

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,обсуждаем,0.441423,0,1,0,0,0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0.469364,0.838754,0.577677,0.0,0.0
1,препарат,0.462957,0,1,0,0,1,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0.469364,0.838754,0.577677,0.0,0.0
2,рисперидон,0.894468,0,1,0,0,2,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,0.978218,0.838754,0.896498,1.0,2.0
3,рисполепт,0.854699,0,1,0,0,3,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,0.428188,0.838754,0.860031,1.0,3.0
4,сперидан,0.803714,0,1,0,0,4,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,0.457736,0.838754,0.811681,1.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9495,временем,0.521677,1,7,1,57,592,inan,ablt,neut,NaN,NOUN,obl,понадобятся,NaN,NaN,NaN,VERB,0.552487,0.484568,0.601818,0.0,0.0
9496,понадобятся,0.486335,1,7,1,57,593,NaN,NaN,NaN,3per,VERB,ROOT,понадобятся,NaN,NaN,NaN,VERB,0.552487,0.484568,0.601818,0.0,0.0
9497,ксф,0.469515,1,7,1,57,594,NaN,NaN,NaN,NaN,NaN,nsubj,понадобятся,NaN,NaN,NaN,VERB,0.552487,0.484568,0.601818,0.0,0.0
9498,но,0.290216,1,7,1,57,595,NaN,NaN,NaN,NaN,CONJ,cc,врача,anim,gent,masc,NOUN,0.393792,0.484568,0.457877,0.0,0.0


In [240]:
df.describe()

,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
count,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000
mean,0.416538,0.904526,3.717895,18.201895,5.843158,1091.000632,0.390562,0.537711,0.504952,0.088737,61.053895
std,0.129465,0.293884,1.607107,15.015207,9.079281,1007.653092,0.099524,0.103919,0.142535,0.392609,340.270862
min,-0.063752,0.000000,1.000000,0.000000,0.000000,0.000000,-0.084358,0.299567,-0.013032,0.000000,0.000000
25%,0.322662,1.000000,3.000000,5.000000,1.000000,242.750000,0.334701,0.465332,0.428370,0.000000,0.000000
50%,0.397888,1.000000,3.000000,15.000000,3.000000,789.000000,0.389719,0.529643,0.511535,0.000000,0.000000
75%,0.479444,1.000000,5.000000,29.000000,7.000000,1769.250000,0.446292,0.588248,0.592360,0.000000,0.000000
max,0.894468,1.000000,7.000000,51.000000,57.000000,3668.000000,0.978218,0.881907,0.925014,3.000000,3653.000000


## **Модели. Обучение, оценка.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [241]:
for_factorize = 'ANIMACY	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS']

In [242]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1, -1])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [243]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'нексиум', 'париет', 'желудке', 'пробиотики', 'схемы', 'обсудить',
        'химиком', 'терапевтом', 'паллиативным', 'понадобятся'],
       dtype='object', length=1795),
 'HEAD_ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'HEAD_

In [244]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [245]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
2,рисперидон,0.894468,0,1,0,0,2,1,0,0,-1,1,2,1,0,0,0,1,0.978218,0.838754,0.896498,1.0,2.0
3,рисполепт,0.854699,0,1,0,0,3,0,1,1,-1,1,3,2,1,0,0,1,0.428188,0.838754,0.860031,1.0,3.0
4,сперидан,0.803714,0,1,0,0,4,-1,-1,0,-1,2,4,3,0,1,1,1,0.457736,0.838754,0.811681,1.0,4.0
21,лекарством,0.364247,0,1,1,1,10,0,5,2,-1,1,4,7,-1,-1,-1,3,0.439526,0.469893,0.534449,2.0,0.0
142,рисполепта,0.774386,0,1,1,13,7,0,0,1,-1,1,21,69,0,2,1,1,0.299209,0.563162,0.718884,1.0,142.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8164,кофеина,0.585196,1,5,49,0,1962,0,1,0,-1,1,4,1572,0,1,1,1,0.380294,0.465332,0.587209,1.0,1966.0
8182,парацетамол,0.728139,1,5,49,2,1980,0,0,0,-1,1,12,1237,-1,-1,-1,0,0.359494,0.371632,0.640772,1.0,1984.0
8191,ибупрофен,0.621706,1,5,49,2,1989,-1,-1,0,-1,15,10,1237,-1,-1,-1,0,0.359494,0.371632,0.640772,1.0,1993.0
8224,ибупрофен,0.621706,1,5,50,0,2022,-1,-1,0,-1,15,12,1237,-1,-1,-1,0,0.359494,0.528033,0.522755,1.0,2026.0


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [246]:
train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 1.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)

In [247]:
train = train.reset_index()

In [248]:
# train = train.iloc[:,:24].drop(['TOKEN', 'level_0', 'index'], axis=1)
# train = train.drop(['TOKEN', 'index'], axis=1)

In [249]:
test = test.reset_index()

In [250]:
test

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,366,препарата,0.360943,1,1,2,5,86,0,1,0,-1,1,21,154,0,1,1,1,0.346080,0.511118,0.440735,2.0,344.0
1,7157,его,0.364522,1,5,30,1,955,-1,3,0,1,5,1,23,-1,-1,-1,0,0.337811,0.653890,0.627462,2.0,937.0
2,7405,нейролептика,0.737105,1,5,36,3,1203,0,1,0,-1,1,21,1456,0,0,2,1,0.395276,0.610779,0.704870,2.0,0.0
3,4725,препараты,0.409828,1,3,33,0,2841,0,0,0,-1,1,4,1010,0,0,0,1,0.433092,0.779981,0.828406,2.0,2853.0
4,7714,лирику,0.709795,1,5,42,2,1512,0,3,1,-1,1,1,447,0,0,1,1,0.325300,0.664678,0.699819,1.0,1516.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,7519,фена,0.629952,1,5,37,4,1317,0,1,0,-1,1,21,1494,0,5,1,1,0.338451,0.529105,0.627789,1.0,1321.0
95,7333,буспирона,0.604997,1,5,35,2,1131,0,1,0,-1,1,4,1456,0,0,2,1,0.395276,0.611564,0.704870,1.0,1135.0
96,7640,нейролептика,0.737105,1,5,40,0,1438,0,1,0,-1,1,10,114,-1,-1,-1,5,0.370802,0.561368,0.718959,2.0,0.0
97,8164,кофеина,0.585196,1,5,49,0,1962,0,1,0,-1,1,4,1572,0,1,1,1,0.380294,0.465332,0.587209,1.0,1966.0


In [251]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [252]:
test.loc[test['NER'] != 0]

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,366,препарата,0.360943,1,1,2,5,86,0,1,0,-1,1,21,154,0,1,1,1,0.346080,0.511118,0.440735,2.0,344.0
1,7157,его,0.364522,1,5,30,1,955,-1,3,0,1,5,1,23,-1,-1,-1,0,0.337811,0.653890,0.627462,2.0,937.0
2,7405,нейролептика,0.737105,1,5,36,3,1203,0,1,0,-1,1,21,1456,0,0,2,1,0.395276,0.610779,0.704870,2.0,0.0
3,4725,препараты,0.409828,1,3,33,0,2841,0,0,0,-1,1,4,1010,0,0,0,1,0.433092,0.779981,0.828406,2.0,2853.0
4,7714,лирику,0.709795,1,5,42,2,1512,0,3,1,-1,1,1,447,0,0,1,1,0.325300,0.664678,0.699819,1.0,1516.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,7519,фена,0.629952,1,5,37,4,1317,0,1,0,-1,1,21,1494,0,5,1,1,0.338451,0.529105,0.627789,1.0,1321.0
95,7333,буспирона,0.604997,1,5,35,2,1131,0,1,0,-1,1,4,1456,0,0,2,1,0.395276,0.611564,0.704870,1.0,1135.0
96,7640,нейролептика,0.737105,1,5,40,0,1438,0,1,0,-1,1,10,114,-1,-1,-1,5,0.370802,0.561368,0.718959,2.0,0.0
97,8164,кофеина,0.585196,1,5,49,0,1962,0,1,0,-1,1,4,1572,0,1,1,1,0.380294,0.465332,0.587209,1.0,1966.0


In [253]:
train.loc[(train['NER'] == 1.0) | (train['NER'] == 2.0 )]

,index,TOKEN,TOKEN_VECT,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,HEAD_VECT,SENT_VECT,HEAD_CHILDS_VECT,NER,COREFERENCE_CLUSTER
0,5987,лекарства,0.310366,1,4,7,1,363,0,3,2,-1,1,4,1202,0,0,0,1,0.447796,0.643862,0.925014,2.0,0.0
1,6051,препарат,0.462957,1,4,7,6,427,0,0,0,-1,1,1,1159,-1,-1,-1,0,0.415188,0.575890,0.589951,2.0,449.0
2,410,сонапакс,0.845896,1,1,4,0,130,0,0,0,-1,1,1,171,-1,-1,-1,5,0.519730,0.627373,0.739339,1.0,410.0
3,1871,зипрекса,0.773054,0,3,0,0,3,1,1,0,-1,1,3,417,0,1,1,1,0.368714,0.777361,0.769416,1.0,3.0
4,5558,галлоперидол,0.876574,0,4,1,0,2,0,0,0,-1,1,1,235,-1,-1,0,0,0.441136,0.533607,0.732384,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,4022,венлафаксин,0.775163,1,3,23,13,2138,1,0,0,-1,1,1,831,-1,1,-1,11,0.338225,0.737903,0.783389,1.0,2154.0
392,3334,зипрексой,0.797749,1,3,18,5,1450,0,5,1,-1,1,10,638,-1,-1,-1,5,0.297711,0.587593,0.779137,1.0,1466.0
393,6539,тиаприд,0.844321,1,5,17,0,337,1,0,0,-1,1,12,1284,-1,-1,-1,0,0.366452,0.497950,0.737240,1.0,341.0
394,6638,катадолон,0.771881,1,5,19,1,436,0,0,0,-1,1,4,274,0,0,0,1,0.326344,0.662388,0.729221,1.0,440.0


In [254]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

Делаем наборы для обучения моделей.

In [255]:
X, y, X_test, y_test = train.drop(['POST_NUM',		'PERSON', 'TOKEN','TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',		'DEPENDENCY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop([ 'POST_NUM',		'PERSON', 'TOKEN','TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',		'DEPENDENCY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [256]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [257]:
X

,TOPIC_NUM,TOKEN_NUM,GENDER,POS,SENT_VECT,NER,COREFERENCE_CLUSTER
0,4,363,2,1,0.643862,2.0,0.0
1,4,427,0,1,0.575890,2.0,449.0
2,1,130,0,1,0.627373,1.0,410.0
3,3,3,0,1,0.777361,1.0,3.0
4,4,2,0,1,0.533607,1.0,5.0
...,...,...,...,...,...,...,...
391,3,2138,0,1,0.737903,1.0,2154.0
392,3,1450,1,1,0.587593,1.0,1466.0
393,5,337,0,1,0.497950,1.0,341.0
394,5,436,0,1,0.662388,1.0,440.0


In [258]:
X_test

,TOPIC_NUM,TOKEN_NUM,GENDER,POS,SENT_VECT,NER,COREFERENCE_CLUSTER
0,1,86,0,1,0.511118,2.0,344.0
1,5,955,0,5,0.653890,2.0,937.0
2,5,1203,0,1,0.610779,2.0,0.0
3,3,2841,0,1,0.779981,2.0,2853.0
4,5,1512,1,1,0.664678,1.0,1516.0
...,...,...,...,...,...,...,...
94,5,1317,0,1,0.529105,1.0,1321.0
95,5,1131,0,1,0.611564,1.0,1135.0
96,5,1438,0,1,0.561368,2.0,0.0
97,5,1962,0,1,0.465332,1.0,1966.0


In [259]:
y

0         0.0
1       449.0
2       410.0
3         3.0
4         5.0
        ...  
391    2154.0
392    1466.0
393     341.0
394     440.0
395    1142.0
Name: COREFERENCE_CLUSTER, Length: 396, dtype: float64

Обучаем модели.

In [260]:
clf = tree.DecisionTreeClassifier(max_depth=2000, max_features='sqrt', )
clf.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=2000, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [261]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=2000, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [262]:
clf.score(X_test, y_test), clf_entropy.score(X_test, y_test)

(0.1111111111111111, 0.25252525252525254)

In [263]:
randf_clf = RandomForestClassifier(max_depth=2000, n_estimators=600, min_samples_split=4, bootstrap=True, )
randf_clf.fit(X, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2000, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [264]:
regr = linear_model.LinearRegression(n_jobs=101)
regr.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=101, normalize=False)

In [265]:
from sklearn import svm

In [266]:
svm_clf = svm.SVC(kernel='rbf', probability=True, decision_function_shape='ovo')
svm_clf.fit(X, y)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [267]:
from sklearn.neighbors import KNeighborsClassifier
knn_clf = KNeighborsClassifier().fit(X, y)

In [268]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


Функция для оценки моделей

In [269]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [270]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [271]:
my_eval(list(naive_pred), list(y_test))

1  true positive
87  false positive
0  true negative
5  false negative
---------
0.011363636363636364  presicion
0.16666666666666666  recall
0.021276595744680854  f-score


In [272]:
my_eval(list(pred_knn), list(y_test))

12  true positive
82  false positive
5  true negative
0  false negative
---------
0.1276595744680851  presicion
1.0  recall
0.22641509433962262  f-score


In [273]:
my_eval(list(pred), list(y_test))

3  true positive
88  false positive
8  true negative
0  false negative
---------
0.03296703296703297  presicion
1.0  recall
0.06382978723404255  f-score


In [274]:
my_eval(list(pred_randf), list(y_test))

5  true positive
86  false positive
8  true negative
0  false negative
---------
0.054945054945054944  presicion
1.0  recall
0.10416666666666666  f-score


In [275]:
my_eval(list(pred_entropy), list(y_test))

17  true positive
74  false positive
8  true negative
0  false negative
---------
0.18681318681318682  presicion
1.0  recall
0.3148148148148148  f-score


In [276]:
my_eval(list(pred_regr), list(y_test)) # Бэйзлайн?

54  true positive
45  false positive
0  true negative
0  false negative
---------
0.5454545454545454  presicion
1.0  recall
0.7058823529411764  f-score


In [277]:
my_eval(list(pred_svm), list(y_test))

2  true positive
42  false positive
8  true negative
15  false negative
---------
0.045454545454545456  presicion
0.11764705882352941  recall
0.06557377049180328  f-score
